# Using Google Geocoding API to get lat and long info for cities

**NOTE TO SELF** this notebook contains an API Key -- fix that before you post

In [18]:
from dotenv import load_dotenv
import os
load_dotenv() 


True

In [19]:
import pandas as pd
import requests

## Testing out the Google GeoCode API to grab the information I will need for all the cities on the Europe board with a single city
Here, we are using Paris as an example

In [23]:
api_key = os.getenv('api_key')
address = 'Paris, France'

In [24]:
params = {
    'key': api_key,
    'address': address
}

url = 'https://maps.googleapis.com/maps/api/geocode/json?'
response = requests.get(url, params=params)
response.json()['results']

[{'address_components': [{'long_name': 'Paris',
    'short_name': 'Paris',
    'types': ['locality', 'political']},
   {'long_name': 'Paris',
    'short_name': 'Paris',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Île-de-France',
    'short_name': 'IDF',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'France',
    'short_name': 'FR',
    'types': ['country', 'political']}],
  'formatted_address': 'Paris, France',
  'geometry': {'bounds': {'northeast': {'lat': 48.9021475, 'lng': 2.4698509},
    'southwest': {'lat': 48.8155622, 'lng': 2.2242191}},
   'location': {'lat': 48.856614, 'lng': 2.3522219},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 48.9021475, 'lng': 2.4698509},
    'southwest': {'lat': 48.8155622, 'lng': 2.2242191}}},
  'place_id': 'ChIJD7fiBh9u5kcRYJSMaMOCCwQ',
  'types': ['locality', 'political']}]

### Grabbing the latitude and longitude from the API request

In [26]:
#lat 
response.json()['results'][0]['geometry']['location']['lat']

#long
response.json()['results'][0]['geometry']['location']['lng']

2.3522219

### Grabbing country name just in case we need that later 


In [27]:
response.json()['results'][0]['formatted_address']

#or, use these to get the full name of the country or the "short name," which I assume is its 2-letter country code 
response.json()['results'][0]['address_components'][3]['long_name']
response.json()['results'][0]['address_components'][3]['short_name']

'FR'

## Now we are going to grab these results for a list of cities and add them to our dataframe


In [28]:
import pandas as pd
import numpy as np 

df = pd.read_csv("tickets.csv")
df.head()

,City A,Formatted City A,City B,Formatted City B,Points,Long_Regular,Category
0,Brest,"Brest, France",Petrograd,"Saint Petersburg, Russia",20,1,Long
1,Lisboa,"Lisbon, Portugal",Danzic,"Gdansk, Poland",20,1,Long
2,Palermo,"Palermo, Italy",Moskva,"Moscow, Russia",20,1,Long
3,Kobenhavn,"Copenhagen, Denmark",Erzurum,"Erzurum, Turkey",21,1,Long
4,Edinburgh,"Edinburgh, Scotland",Athína,"Athens, Greece",21,1,Long


In [29]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

### Here, we are testing the API request by converting the city_a column to a list and running it through the API 


In [30]:
cities_a = df['city_a'].tolist()
for city in cities_a[:3]: 
    address = city 
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(url, params=params)
    print(response.json()['results'][0]['geometry']['location']['lat'])
    print(address)
    print('----')

48.856614
Brest
----
48.856614
Lisboa
----
48.856614
Palermo
----


## Coverting my dataframe *BACK* into a list of dictionaries, so I can more easily add the new values I want: 
   * lat
   * long
   * country
   * country code (just in case)

In [31]:
tickets = df.to_dict('records') 
#tickets

In [32]:
# for ticket in tickets: 
#     print(ticket['formatted_city_a'])
#     print('----')
#     print(ticket['formatted_city_b'])
#     print('----')


## Running the API request for all origin cities

In [34]:
#Here, we are appending the lat and long for every city_a

for ticket in tickets: 
    address = ticket['formatted_city_a']
    api_key = os.getenv('api_key')
    params = {
    'key': api_key,
    'address': address}
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(url, params=params)
    data = response.json()['results'][0]
    try: 
        ticket['lat_city_a'] = data['geometry']['location']['lat']
        ticket['long_city_a'] = data['geometry']['location']['lng']
#         print(address)
#         print(ticket['long_city_a'])
#         print(ticket['lat_city_a'])
#         print('---')
    except: 
#        print('problem!!')
        pass
    



## Running the API request for all destination cities

In [35]:
#And here, we are appending the lat and long for every city_b


for ticket in tickets: 
    address = ticket['formatted_city_b']
#    print(address)
    api_key = os.getenv('api_key')
    params = {
    'key': api_key,
    'address': address}
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(url, params=params)
    data = response.json()['results'][0]
    ticket['lat_city_b'] = data['geometry']['location']['lat']
    ticket['long_city_b'] = data['geometry']['location']['lng']
#     print(address)
#     print(ticket['lat_city_b'])
#     print(ticket['long_city_b'])
#     print('---')


In [36]:
df = pd.DataFrame(tickets)

### And now we have our new df with location data! 


In [38]:
df.head()

,city_a,formatted_city_a,city_b,formatted_city_b,points,long_regular,category,lat_city_a,long_city_a,lat_city_b,long_city_b
0,Brest,"Brest, France",Petrograd,"Saint Petersburg, Russia",20,1,Long,48.390394,-4.486076,59.931058,30.360910
1,Lisboa,"Lisbon, Portugal",Danzic,"Gdansk, Poland",20,1,Long,38.722252,-9.139337,54.352025,18.646638
2,Palermo,"Palermo, Italy",Moskva,"Moscow, Russia",20,1,Long,38.115690,13.361487,55.755826,37.617300
3,Kobenhavn,"Copenhagen, Denmark",Erzurum,"Erzurum, Turkey",21,1,Long,55.676097,12.568337,39.905499,41.265824
4,Edinburgh,"Edinburgh, Scotland",Athína,"Athens, Greece",21,1,Long,55.953252,-3.188267,37.983810,23.727539


In [39]:
#df.to_csv('tickets_latlong.csv', index=False)

# Applying the same API request for to Europe_city_info.csv

In [43]:
import pandas as pd
import requests
df = pd.read_csv('Europe_city_info_formatted.csv')
df = df.drop(['train_routes_per_city', 'tunnel_count','ferry_route_count', 'city'], 1)
cities = df.to_dict('records') 


/var/folders/qx/nkh2pq6x3_193757ddxvrcf40000gp/T/ipykernel_18210/4227922560.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(['train_routes_per_city', 'tunnel_count','ferry_route_count', 'city'], 1)


In [44]:

for city in cities: 
    address = city['city_formatted']
    api_key = os.getenv('api_key')
    params = {
    'key': api_key,
    'address': address}
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(url, params=params)
    data = response.json()['results'][0]
    city['lat_city'] = data['geometry']['location']['lat']
    city['long_city'] = data['geometry']['location']['lng']
#     print(address)
#     print(city['lat_city'])
#     print(city['long_city'])
#     print('---')



In [45]:
df = pd.DataFrame(cities)


In [46]:
#df.to_csv('cities.csv', index=False)

# Now feeding routes.csv into API 

In [47]:
df_routes = pd.read_csv('routes.csv')
df_routes.columns = df_routes.columns.str.lower().str.replace(' ', '_')
df_routes.head()
routes = df_routes.to_dict('records')


In [47]:
for route in routes: 
    address = route['city_a_formatted']
    api_key = os.getenv('api_key')
    params = {
    'key': api_key,
    'address': address}
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(url, params=params)
    data = response.json()['results'][0]
    route['lat_city_a'] = data['geometry']['location']['lat']
    route['long_city_a'] = data['geometry']['location']['lng']

In [48]:
for route in routes: 
    address = route['city_b_formatted']
    api_key = os.getenv('api_key')
    params = {
    'key': api_key,
    'address': address}
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(url, params=params)
    data = response.json()['results'][0]
    route['lat_city_b'] = data['geometry']['location']['lat']
    route['long_city_b'] = data['geometry']['location']['lng']

In [50]:
df = pd.DataFrame(routes)

In [51]:
#df.to_csv('routes.csv', index=False)